#### __2. 실습 환경 설정과 파이토치 기초__

__2.2 파이토치 기초 문법__

In [1]:
# 텐서 생성 및 변환
import torch
print(torch.tensor([[1, 2], [3, 4]]))
print(torch.tensor([[1, 2], [3, 4]], device="cuda:0"))  # GPU에 텐서 생성
print(torch.tensor([[1, 2], [3, 4]], dtype=torch.float64))

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]], device='cuda:0')
tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)


In [2]:
temp = torch.tensor([[1, 2], [3, 4]])
print(temp.numpy())  # 텐서를 ndarray로 변환

[[1 2]
 [3 4]]


In [3]:
temp = torch.tensor([[1, 2], [3, 4]], device="cuda:0")
print(temp.to("cpu").numpy())  # GPU상의 텐서를 CPU의 텐서로 변환 후 ndarray로 변환

[[1 2]
 [3 4]]


In [4]:
# 인덱스 조작
temp = torch.FloatTensor([1, 2, 3, 4, 5, 6, 7])
print(temp[0], temp[1], temp[-1])
print(temp[2:5], temp[4:-1])

tensor(1.) tensor(2.) tensor(7.)
tensor([3., 4., 5.]) tensor([5., 6.])


In [6]:
# 텐서 연산 및 차원 조작
v = torch.tensor([1, 2, 3])
w = torch.tensor([3, 4, 6])
print(w - v)

tensor([2, 2, 3])


In [7]:
temp = torch.tensor([
    [1, 2], [3, 4]])

print(temp.shape)

torch.Size([2, 2])


In [8]:
print(temp.view(4, 1))  # 2x2행렬을 4x1로 변형

tensor([[1],
        [2],
        [3],
        [4]])


In [9]:
print(temp.view(-1))  # 2x2행렬을 1차원 벡터로 변형

tensor([1, 2, 3, 4])


In [10]:
print(temp.view(1, -1))  # -1은 (1, ?)와 같은 의미, 다른 차원으로부터 값을 유추

tensor([[1, 2, 3, 4]])
